In [ ]:
class Discriminator_img(nn.Module):
    def __init__(self):
        super(Discriminator_img,self).__init__()
        
        self.conv1 = nn.Sequential(nn.Conv2d(6,64,kernel_size=(4,4),padding=(1,1),stride=(2,2),bias=False),
                                   nn.BatchNorm2d(64),
                                   nn.LeakyReLU(0.15, inplace=True)
                                  )
        self.conv2 = nn.Sequential(nn.Conv2d(64,128,kernel_size=(4,4),padding=(1,1),stride=(2,2),bias=False),
                                   nn.BatchNorm2d(128),
                                   nn.LeakyReLU(0.15, inplace=True)
                                  )
        self.conv3 = nn.Sequential(nn.Conv2d(128,256,kernel_size=(4,4),padding=(1,1),stride=(2,2),bias=False),
                                   nn.BatchNorm2d(256),
                                   nn.LeakyReLU(0.15, inplace=True)
                                  )
        self.conv4 = nn.Sequential(nn.Conv2d(256,512,kernel_size=(4,4),padding=(1,1),stride=(2,2),bias=False),
                                   nn.BatchNorm2d(512),
                                   nn.LeakyReLU(0.15, inplace=True)
                                  )
        self.conv5 = nn.Sequential(nn.Conv2d(512,1024,kernel_size=(4,4),padding=(1,1),stride=(2,2),bias=False),
                                   nn.BatchNorm2d(1024),
                                   nn.LeakyReLU(0.15, inplace=True)
                                  )    
        self.conv6 = nn.Sequential(nn.Conv2d(1024,512,kernel_size=(3,3),padding=(1,1),stride=(2,2),bias=False),
                                   nn.BatchNorm2d(512),
                                   nn.LeakyReLU(0.15, inplace=True)
                                  )    
        self.conv7 = nn.Conv2d(512,1,kernel_size=(3,3),stride=(1,1),bias=False)
                                  
        self.sig = nn.Sigmoid()
    def forward(self,x,x1): 
        #print(x.shape)
        #print(x1.shape)
        x = torch.cat((x,x1),1)
        x = self.conv1(x)
        #print(x.shape)
        x = self.conv2(x)
        #print(x.shape)
        x = self.conv3(x)
        #print(x.shape)
        x = self.conv4(x)
        #print(x.shape)
        x = self.conv5(x)
        #print(x.shape)
        x = self.conv6(x)
        x = self.conv7(x) 
        x = self.sig(x)
        #print(x)
        return x
    def save(self, PATH):
        torch.save(self.state_dict(), PATH)

    def load(self, PATH):
        self.load_state_dict(torch.load(PATH))

In [ ]:
class Discriminator_seq(nn.Module):
    def __init__(self):
        super(Discriminator_seq,self).__init__()
        self.audencoder = AudioEncode()
       
        self.conv1 = nn.Sequential(nn.Conv2d(3,64,kernel_size=(4,4),padding=(1,1),stride=(2,2), bias=False),
                                   nn.BatchNorm2d(64),
                                   nn.LeakyReLU(0.1, inplace=True)
                                  )
        self.conv2 = nn.Sequential(nn.Conv2d(64,128,kernel_size=(4,4),padding=(1,1),stride=(2,2), bias=False),
                                   nn.BatchNorm2d(128),
                                   nn.LeakyReLU(0.1, inplace=True)
                                  )
        self.conv3 = nn.Sequential(nn.Conv2d(128,256,kernel_size=(4,4),padding=(1,1),stride=(2,2), bias=False),
                                   nn.BatchNorm2d(256),
                                   nn.LeakyReLU(0.1, inplace=True)
                                  )
        self.conv4 = nn.Sequential(nn.Conv2d(256,512,kernel_size=(4,4),padding=(1,1),stride=(2,2), bias=False),
                                   nn.BatchNorm2d(512),
                                   nn.LeakyReLU(0.1, inplace=True)
                                  )
        self.conv5 = nn.Sequential(nn.Conv2d(512,1024,kernel_size=(4,4),padding=(1,1),stride=(2,2), bias=False),
                                   nn.BatchNorm2d(1024),
                                   nn.LeakyReLU(0.1, inplace=True)
                                  )    
        self.conv6 = nn.Sequential(nn.Conv2d(1024,256,kernel_size=(6,6),stride=(1,1), bias=False), 
                                   nn.BatchNorm2d(256),
                                   nn.LeakyReLU(0.1, inplace=True)
                                  )
        self.fc = nn.Linear(320,1,bias=False)
        self.Gru = nn.GRU(256,64,2,batch_first=True)
        self.sig = nn.Sigmoid()
        
    def forward(self,x,z):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.conv6(x)
        #print(x.shape)
        x = x.view(x.shape[0],1,-1)
        #print(x.shape)
        x,_ = self.Gru(x)
        #print(x.shape)
        _,z = self.audencoder(z) 
        x = x.view(x.shape[0],-1)
        x = torch.cat((x,z),1)  
        #print(x.shape)
        #x = x.view(x.shape[0],-1)
        x = self.fc(x)
        x = self.sig(x)
        #x = self.relu(x)
        return x
    def save(self,PATH):
        torch.save(self.state_dict(), PATH)

    def load(self, PATH):
        self.load_state_dict(torch.load(PATH))